## Prep to run this notebook:
- scripts/run_vk_ref_geuvadis.py
- scripts/run_vk_count_geuvadis.py
- scripts/run_vk_count_geuvadis_wxs.py

In [ ]:
# try:
#     import varseek as vk
# except ImportError:
#     print("varseek not found, installing...")
#     !pip install -U -q varseek
# try:
#     import RLSRWP_2025
# except ImportError:
#     print("RLSRWP_2025 not found, installing...")
#     !pip install -q git+https://github.com/pachterlab/RLSRWP_2025.git

In [ ]:
from matplotlib_venn import venn2
import matplotlib.pyplot as plt
import os
import pandas as pd
import numpy as np
import seaborn as sns
import re
import anndata as ad
import scanpy as sc

import varseek as vk
from RLSRWP_2025.constants import box_links_dict

RLSRWP_2025_dir = os.path.dirname(os.path.abspath(""))  # if this notebook resides in RLSRWP_2025/notebooks/0_data_download.ipynb, then this retrieves RLSRWP_2025
data_dir = os.path.join(RLSRWP_2025_dir, "data")

In [ ]:
sequencing_data_out_base = os.path.join(data_dir, "geuvadis_data_base")
geuvadis_reference_files_dir = os.path.join(data_dir, "reference", "geuvadis")
variants_transcriptome_df_path = os.path.join(geuvadis_reference_files_dir, "variants_transcriptome.parquet")
reference_genome_t2g = os.path.join(data_dir, "reference", "ensembl_grch37_release113", "t2g.txt")

geuvadis_genotype_true_adata = os.path.join(geuvadis_reference_files_dir, "genotypes_adata_true.h5ad")
adata_combined_path_reference_genome = os.path.join(sequencing_data_out_base, "adata_reference_genome_combined.h5ad")
adata_combined_path_vcrs_final = os.path.join(sequencing_data_out_base, f"adata_vcrs_combined_w37_k41.h5ad")

w_and_k_list_of_dicts = [
    {"w": 27, "k": 31},
    {"w": 37, "k": 41},
    {"w": 47, "k": 51},
]

min_counts = 2
min_samples_per_variant = 3
n_top_variants = 2000
gene_set_to_exclude = {'ENSG00000111640', 'ENSG00000242580', 'ENSG00000243466', 'ENST00000603632', 'ENST00000605760'}  # None, or a set of gene names to exclude from the analysis (in ENSGs)

out_dir = os.path.join(data_dir, f"geuvadis_analysis_total")

In [ ]:
if not os.path.isfile(adata_combined_path_reference_genome):
    vk.utils.download_box_url(box_links_dict["geuvadis_adata_reference_genome"], output_file_name=adata_combined_path_reference_genome)
if not os.path.isfile(geuvadis_genotype_true_adata):
    vk.utils.download_box_url(box_links_dict["geuvadis_adata_genotype"], output_file_name=geuvadis_genotype_true_adata)
if not os.path.isfile(adata_combined_path_vcrs_final):
    vk.utils.download_box_url(box_links_dict[f"geuvadis_adata_vcrs_w37_k41"], output_file_name=adata_combined_path_vcrs_final)

adata_reference = ad.read_h5ad(adata_combined_path_reference_genome)
adata_genotype = ad.read_h5ad(geuvadis_genotype_true_adata)
adata_vcrs = ad.read_h5ad(adata_combined_path_vcrs_final)

os.makedirs(out_dir, exist_ok=True)

In [ ]:
if gene_set_to_exclude is not None:
    adata_vcrs = adata_vcrs[:, ~adata_vcrs.var['transcript_ID'].isin(gene_set_to_exclude).values]
    adata_vcrs = adata_vcrs[:, ~adata_vcrs.var['gene'].isin(gene_set_to_exclude).values]
    adata_genotype = adata_genotype[:, ~adata_genotype.var['transcript_ID'].isin(gene_set_to_exclude).values]
    adata_genotype = adata_genotype[:, ~adata_genotype.var['gene'].isin(gene_set_to_exclude).values]

## Each sample is a "cell", and each variant is a "gene"

In [ ]:
sc.pp.filter_genes(adata_vcrs, min_cells=min_samples_per_variant)

In [ ]:
sc.pp.highly_variable_genes(adata_vcrs, n_top_genes=n_top_variants, batch_key="Population name")

In [ ]:
sc.tl.pca(adata_vcrs)

In [ ]:
sc.pp.scale(adata_vcrs, max_value=None)

## Let's hope for some juicy clusters!

In [ ]:
sc.pl.pca_variance_ratio(adata_vcrs, n_pcs=50, log=True)

sc.pl.pca(
    adata_vcrs,
    color=["Superpopulation name"],
    dimensions=[(0, 1)],
    ncols=2,
    size=2,
)